In [4]:
import pandas as pd
import tensorflow as tf

In [1]:
with open('./data/Sonnet.txt', 'r') as file:
    sonnets = file.read()

In [2]:
sonnets = sonnets.replace('\n', ' ').replace('\r', '')
sonnets = sonnets.split('\n\n')

In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sonnets)
sequences = tokenizer.texts_to_sequences(sonnets)
vocab_size = len(tokenizer.word_index) + 1
max_length = max([len(seq) for seq in sequences])
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, padding='post')

In [6]:
latent_dim = 100

generator = tf.keras.models.Sequential()
generator.add(tf.keras.layers.Dense(256, input_dim=latent_dim, activation='leaky_relu'))
generator.add(tf.keras.layers.Dense(512, activation = 'leaky_relu'))
generator.add(tf.keras.layers.Dense(1024, activation='leaky_relu'))
generator.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))

In [7]:
discriminator = tf.keras.models.Sequential()
discriminator.add(tf.keras.layers.Embedding(vocab_size, 100, input_length=max_length))
discriminator.add(tf.keras.layers.Conv1D(64, 8, padding='same', activation='leaky_relu'))
discriminator.add(tf.keras.layers.Conv1D(128, 8, padding='same', activation='leaky_relu'))
discriminator.add(tf.keras.layers.MaxPooling1D())
discriminator.add(tf.keras.layers.Conv1D(128, 8, padding='same', activation='leaky_relu'))
discriminator.add(tf.keras.layers.Conv1D(256, 8, padding='same', activation='leaky_relu'))
discriminator.add(tf.keras.layers.MaxPooling1D())
discriminator.add(tf.keras.layers.Flatten())
discriminator.add(tf.keras.layers.Dense(1, activation='sigmoid'))


In [ ]:
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
discriminator.trainable = False
gan_input = Input(shape=(latent_dim,))
generated_sonnet = generator(gan_input)
gan_output = discriminator(generated_sonnet)
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy